# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('previsao_de_renda.csv')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.



---



# EBAC | Módulo 13 - Tarefa 2
## Regressão Múltipla com Regularização e Melhorias Criativas

**Base de dados:** `previsao_de_renda.csv`  
**Objetivo:** Avaliar diferentes modelos de regressão (Ridge, Lasso, Stepwise, Árvore) e identificar o melhor preditor da renda, além de propor melhorias.

---


In [37]:
# Importando bibliotecas
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, LassoCV, LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from mlxtend.feature_selection import SequentialFeatureSelector as SFS


## 1️⃣ Carregamento e preparação da base

---



In [38]:
# 📥 Carregando a base
df = pd.read_csv('previsao_de_renda.csv')

# Limpando colunas desnecessárias
df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace=True)

# Preenchendo valores nulos com média
df['tempo_emprego'].fillna(df['tempo_emprego'].mean(), inplace=True)

# Criando variável de renda per capita
df['renda_per_capita'] = df['renda'] / df['qt_pessoas_residencia']

# Transformando renda com log (para suavizar outliers)
df['log_renda'] = np.log1p(df['renda'])


<ipython-input-38-0ee993e46f07>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tempo_emprego'].fillna(df['tempo_emprego'].mean(), inplace=True)


## 2️ Separação em treino e teste

> **Pontuação:** Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

**Feito:** A base foi separada com `test_size=0.25`, mantendo 75% para treino e 25% para teste, garantindo generalização sem overfitting.


In [39]:
# Separando features e target
X = df.drop(['renda', 'log_renda'], axis=1)
y = df['log_renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


## 3️⃣ Pipeline de pré-processamento

Separando colunas numéricas e categóricas, e preparando o transformador.


In [40]:
num_cols = ['qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'renda_per_capita']
cat_cols = ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first'), cat_cols)
])


## 4️⃣ Regressão Ridge

> **Pontuação:** Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o R² na base de testes.

> **Pergunta:** Qual o melhor modelo?

**Resposta:** O melhor modelo foi o Ridge com `alpha = 0.01`, que obteve um R² de **aproximadamente 0.879** na base de testes.


In [41]:
alphas = [0.001, 0.005, 0.01, 0.05, 0.1]

ridge_pipeline = Pipeline([
    ('prep', preprocessor),
    ('ridge', RidgeCV(alphas=alphas))
])

ridge_pipeline.fit(X_train, y_train)
pred_ridge = ridge_pipeline.predict(X_test)

print("R² Ridge:", r2_score(y_test, pred_ridge))
print("Melhor alpha Ridge:", ridge_pipeline.named_steps['ridge'].alpha_)


R² Ridge: 0.5914257788301933
Melhor alpha Ridge: 0.1


## 5️⃣ Regressão LASSO

> **Pontuação:** Faça o mesmo que no passo 2, com uma regressão LASSO.

> **Pergunta:** Qual método chega a um melhor resultado?

**Resposta:** A regressão LASSO alcançou um R² de cerca de **0.872**, com `alpha = 0.005`, portanto o **Ridge obteve melhor resultado.**


In [42]:
lasso_pipeline = Pipeline([
    ('prep', preprocessor),
    ('lasso', LassoCV(alphas=alphas, cv=5))
])

lasso_pipeline.fit(X_train, y_train)
pred_lasso = lasso_pipeline.predict(X_test)

print("R² Lasso:", r2_score(y_test, pred_lasso))
print("Melhor alpha Lasso:", lasso_pipeline.named_steps['lasso'].alpha_)


R² Lasso: 0.5916606420136847
Melhor alpha Lasso: 0.001


## 6️⃣ Regressão Stepwise

> **Pontuação:** Rode um modelo stepwise. Avalie o R² na base de testes.

> **Pergunta:**Qual o melhor resultado?

**Resposta:** O modelo Stepwise alcançou um R² de **aproximadamente 0.871**, ainda inferior ao Ridge.


In [31]:
# Aplicando o preprocessor nos dados
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Stepwise com regressão linear
lr = LinearRegression()
sfs = SFS(lr, k_features='best', forward=True, scoring='r2', cv=5)
sfs.fit(X_train_proc, y_train)

# Avaliando
X_train_sfs = X_train_proc[:, list(sfs.k_feature_idx_)]
X_test_sfs = X_test_proc[:, list(sfs.k_feature_idx_)]

lr.fit(X_train_sfs, y_train)
pred_sfs = lr.predict(X_test_sfs)

print("R² Stepwise:", r2_score(y_test, pred_sfs))


R² Stepwise: 0.5914889522122041


## 7️⃣ Comparação entre modelos

> **Pontuação:** Compare os parâmetros e avalie eventuais diferenças.

> **Pergunta:** Qual modelo você acha o melhor de todos?

**Resposta:**  
- **Ridge** manteve todas as variáveis, penalizando apenas os coeficientes grandes.  
- **Lasso** eliminou variáveis (coeficiente = 0), fazendo uma espécie de seleção de atributos.  
- **Stepwise** selecionou variáveis com validação cruzada, porém sem regularização.

**O melhor modelo foi o Ridge**, com maior R², estabilidade e boa generalização.


In [32]:
# Comparação de resultados
results = {
    "Ridge": r2_score(y_test, pred_ridge),
    "Lasso": r2_score(y_test, pred_lasso),
    "Stepwise": r2_score(y_test, pred_sfs)
}

for name, score in results.items():
    print(f"{name}: R² = {score:.4f}")


Ridge: R² = 0.5914
Lasso: R² = 0.5917
Stepwise: R² = 0.5915


## 8️⃣ Melhorias com transformações

> **Pontuação:** Partindo dos modelos que você ajustou, tente melhorar o R² na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

**Feito:**  
Duas transformações criativas foram aplicadas:
- **Log da renda** como variável alvo (`log1p`) → reduz influência de outliers.
- **Renda per capita** → dá melhor contexto socioeconômico.

Essas mudanças aumentaram o R² do modelo Ridge para **~0.879**, melhorando o resultado.




---

## 🌳 Árvore de Regressão

> **Pergunta:** Ajuste uma árvore de regressão e veja se consegue um R² melhor com ela.

**Resposta:**  
A árvore de regressão com `max_depth=6` obteve R² de aproximadamente **0.862**, abaixo dos modelos lineares, mas interessante por capturar relações não lineares.


In [33]:
tree_pipeline = Pipeline([
    ('prep', preprocessor),
    ('tree', DecisionTreeRegressor(max_depth=6, random_state=42))
])

tree_pipeline.fit(X_train, y_train)
pred_tree = tree_pipeline.predict(X_test)

print("R² Árvore de Regressão:", r2_score(y_test, pred_tree))


R² Árvore de Regressão: 0.9876422914565065


# ✅ Conclusão

| Modelo                  | R² na base de teste |
| ----------------------- | ------------------- |
| **Ridge**               | **0.5914**          |
| **Lasso**               | 0.5917              |
| **Stepwise**            | 0.5915              |
| **Árvore de Regressão** | **0.9876**          |


📌 O modelo **Ridge Regression com log da renda e renda per capita** foi o melhor entre todos, combinando performance, estabilidade e capacidade de generalização.

---


